# Project Goal

Characterize materials using ultrasound.
Goal is to analyze and characterize materials groups such as plastics, composites and metals.




## Oscillation Frequency Choice

In [ ]:
# Container for fetched data
materials_data = {}

##########################################################################################
######################## The Materials Project imports ###################################
##########################################################################################

import sys
import os
sys.path.insert(0, os.path.abspath("") + '/ae_secrets')
from mp_api.client import MPRester
import numpy as np
from mq_secrets import API_KEY
# Replace with your actual API key
from pymatgen.ext.matproj import MPRester
from pprint import pprint

#! Materials
categories = {
    "PCB_materials": {
        # FR4 is a glass-reinforced epoxy laminate;
        # often represented by its main constituents
        "FR4_glass": "mp-8352",         # Glass fiber
        "Alumina_ceramic": "mp-1143"   # Common ceramic
    },
    "Metals": {
        "Copper": "mp-30", 
        "Tin": "mp-117",
        "Iron": "mp-13"
	}
}


fields=["density", "bulk_modulus", "shear_modulus", "material_id"]

# Fetch data using the mp-api client’s summary.search endpoint
with MPRester(API_KEY) as mpr:
	# Get PCB materials
	mlist = list(categories["PCB_materials"].values()) + list(categories["Metals"].values())
	mkeys = list(categories["PCB_materials"].keys()) + list(categories["Metals"].keys())
	print(mkeys)
	docs = mpr.materials.summary.search(material_ids=mlist)
	for i, d in enumerate(docs):
		rho = d['density']
		K = d['bulk_modulus']['vrh'] # GPa
		G = d['shear_modulus']['vrh'] # GPa
		print(f"{mkeys[i]} - K: {d['bulk_modulus']}, G: {d['shear_modulus']}, rho: {rho}")
		print(d["elastic_tensor"])
		## convert GPa→Pa, g/cm³→kg/m³ for SI:
		rho_si = rho * 1e3  
		K_si = K * 1e9
		G_si = G * 1e9
		print(mkeys[i])
		materials_data[mkeys[i]] = {}
		materials_data[mkeys[i]]['rho_si'] = rho_si
		materials_data[mkeys[i]]['K_si'] = K_si
		materials_data[mkeys[i]]['G_si'] = G_si

##########################################################################################
############################## Manual findings ###########################################
##########################################################################################

aluminium_data = {'rho_si': (2.57+2.95)/2, 'K_si': ((62+106) / 2)*1.0e9, 'G_si': ((25+34) / 2) * 1.0e9}
aluminium_data = {'rho_si': (2.57+2.95)/2, 'K_si': ((62+106) / 2)*1.0e9, 'G_si': ((25+34) / 2) * 1.0e9}

##########################################################################################
############################## Data Computation ##########################################
##########################################################################################


for keys, value in materials_data.items():
		K_si = materials_data['K_si']
		G_si = materials_data['G_si']
		rho_si = materials_data['rho_si']
		## longitudinal speed: sqrt((K + 4/3 G)/ρ)
		v_l = np.sqrt((K_si + 4/3*G_si)/rho_si)
		## shear speed: sqrt(G/ρ)
		v_s = np.sqrt(G_si/rho_si)
		materials_data[mkeys[i]]['v_l'] = v_l
		materials_data[mkeys[i]]['v_s'] = v_s
pprint(materials_data)

['FR4_glass', 'Alumina_ceramic', 'Copper', 'Tin', 'Iron']
FR4_glass - K: {'voigt': 146.005, 'reuss': 146.005, 'vrh': 146.005}, G: {'voigt': 59.854, 'reuss': 46.959, 'vrh': 53.406}, rho: 1.970734329777291


KeyError: 'elastic_tensor'

In [ ]:
# Material Properties


In [ ]:
# Penetration Depth


### Conclusion

## Piezo-electric transducer availability
- Acoustic impedance check
- Optional matching material check